<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [27]:
import numpy as np
import pandas as pd
import gensim
import os
import re
import sys

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore

In [6]:
!unzip '/Users/Elizabeth/DS-Unit-4-Sprint-1-NLP/module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip'

Archive:  /Users/Elizabeth/DS-Unit-4-Sprint-1-NLP/module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv.zip
  inflating: Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv  
   creating: __MACOSX/
  inflating: __MACOSX/._Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv  


In [2]:
path = './Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv'

In [81]:
amazon = pd.read_csv(path)


def wrangle(df):
    df['reviews.date'] = [i.split('T')[0] for i in df['reviews.date']]
    df['reviews.dateSeen'] = [i.split('T')[0] for i in df['reviews.dateSeen']]
    
    drop_columns = ['reviews.sourceURLs', 'sourceURLs', 'reviews.numHelpful', 'reviews.id'
                    , 'reviews.doRecommend', 'reviews.didPurchase', 'keys', 'imageURLs', 'asins'
                    , 'dateAdded', 'dateUpdated', 'manufacturerNumber', 'id', 'categories' 
                    ,'reviews.username', 'manufacturer']
    df.drop(columns=drop_columns, inplace=True)
    return df

amazon = wrangle(amazon)
amazon.head(3)

,name,brand,primaryCategories,reviews.date,reviews.dateSeen,reviews.rating,reviews.text,reviews.title
0,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,Health & Beauty,2017-03-02,2017-08-28,3,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...
1,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,Health & Beauty,2016-08-31,2017-08-28,4,Bulk is always the less expensive way to go fo...,... always the less expensive way to go for pr...
2,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,Health & Beauty,2016-12-06,2017-08-28,5,Well they are not Duracell but for the price i...,... are not Duracell but for the price i am ha...


In [86]:
df = amazon
df['reviews.text'].value_counts(normalize=True)[:50]

good                0.002118
great               0.001729
Good                0.001482
Great               0.001341
ok                  0.001165
Great price         0.001059
great price         0.000988
Excellent           0.000988
Great value         0.000953
good price          0.000812
great value         0.000777
Great price!        0.000635
good value          0.000600
Great value.        0.000565
good deal           0.000565
OK                  0.000529
Great product       0.000529
Good deal           0.000529
Great deal          0.000529
As expected         0.000529
Good batteries      0.000494
Good value          0.000494
great deal          0.000459
So far so good      0.000459
Great batteries.    0.000459
Great value!        0.000459
Good price          0.000459
Work great          0.000424
Good buy            0.000424
As described        0.000388
Great price.        0.000388
Good value.         0.000388
Great deal!         0.000388
good batteries      0.000388
very good     

In [87]:
import re
def tokenize(text):
    """Parses a string into a list of semantic units (words)
    
    Args:
        text (str): The string that the function will tokenize.

    Returns:
        list: tokens parsed out by the mechanics of your choice
    """
    tokens = re.sub(r'[^a-zA-Z ^0-9]', '', text)
    tokens = tokens.lower().split()
    
    return tokens

df['tokens'] = [tokenize(i) for i in df['reviews.text']]

In [88]:
df['tokens'].head()

0    [i, order, 3, of, them, and, one, of, the, ite...
1    [bulk, is, always, the, less, expensive, way, ...
2    [well, they, are, not, duracell, but, for, the...
3    [seem, to, work, as, well, as, name, brand, ba...
4    [these, batteries, are, very, long, lasting, t...
Name: tokens, dtype: object

In [89]:
df['reviews.title'][0]

'... 3 of them and one of the item is bad quality. Is missing backup spring so I have ...'

In [90]:
titles = [t[4:] for t in df['brand']]

In [91]:
id2word = corpora.Dictionary(df['tokens'])

In [98]:
id2word.token2id['amazon']

57

In [99]:
sys.getsizeof(id2word)

56

In [100]:
len(id2word.keys())

12895

In [101]:
id2word.filter_extremes(no_below=20, no_above=0.85)

In [102]:
len(id2word.keys())

1866

In [103]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [104]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=42,
                   num_topics = 15,
                   passes=10,
                   workers=2
                  )

In [105]:
lda.print_topics()

[(0,
  '0.061*"the" + 0.052*"it" + 0.036*"a" + 0.036*"to" + 0.030*"i" + 0.028*"is" + 0.022*"this" + 0.021*"and" + 0.018*"as" + 0.016*"you"'),
 (1,
  '0.130*"tablet" + 0.087*"so" + 0.057*"for" + 0.048*"good" + 0.047*"far" + 0.033*"this" + 0.031*"no" + 0.029*"a" + 0.028*"kids" + 0.025*"with"'),
 (2,
  '0.091*"i" + 0.059*"it" + 0.047*"was" + 0.035*"a" + 0.034*"my" + 0.033*"and" + 0.033*"the" + 0.033*"for" + 0.032*"this" + 0.022*"with"'),
 (3,
  '0.092*"the" + 0.045*"is" + 0.043*"buy" + 0.040*"best" + 0.035*"quality" + 0.033*"good" + 0.032*"and" + 0.031*"price" + 0.025*"great" + 0.024*"battery"'),
 (4,
  '0.049*"i" + 0.045*"not" + 0.041*"to" + 0.036*"a" + 0.033*"tablet" + 0.032*"it" + 0.031*"what" + 0.029*"do" + 0.029*"but" + 0.025*"does"'),
 (5,
  '0.051*"you" + 0.047*"to" + 0.036*"the" + 0.029*"have" + 0.026*"a" + 0.021*"it" + 0.020*"be" + 0.019*"if" + 0.016*"will" + 0.016*"that"'),
 (6,
  '0.081*"i" + 0.042*"kindle" + 0.042*"my" + 0.037*"it" + 0.035*"and" + 0.035*"one" + 0.032*"the" + 0

In [106]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
topics = [' '.join(t[0:10]) for t in words]
for t in topics: 
    print(t)
    print("\n")

the it a to i is this and as you


tablet so for good far this no a kids with


i it was a my and the for this with


the is buy best quality good and price great battery


i not to a tablet it what do but does


you to the have a it be if will that


i kindle my it and one the to love had


the is and a fire screen to kindle this than


i the a of in and to batteries these have


to and the tablet easy use apps for is of


it for my and she loves this to old her


they and them work are for these great love kids


batteries as long last these the they brand just name


the it for is and a kids i to that


great price good for a product works and value recommend




In [107]:
lda[corpus[0]]

[(3, 0.16716293), (8, 0.80295193)]

In [108]:
distro = [lda[d] for d in corpus]

def update(doc):
        d_dist = {k:0 for k in range(0,15)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [117]:
df = pd.DataFrame.from_records(new_distro, index=titles)
df.columns = topics
df['brand'] = df.reset_index()['index'].apply(lambda x: x).tolist()

In [118]:
df.head()

,the it a to i is this and as you,tablet so for good far this no a kids with,i it was a my and the for this with,the is buy best quality good and price great battery,i not to a tablet it what do but does,you to the have a it be if will that,i kindle my it and one the to love had,the is and a fire screen to kindle this than,i the a of in and to batteries these have,to and the tablet easy use apps for is of,it for my and she loves this to old her,they and them work are for these great love kids,batteries as long last these the they brand just name,the it for is and a kids i to that,great price good for a product works and value recommend,brand
onbasics,0.0,0.0,0.000000,0.167180,0.0,0.0,0.0,0.0,0.802935,0.0,0.0,0.0,0.000000,0.0,0.0,onbasics
onbasics,0.0,0.0,0.000000,0.265772,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.672323,0.0,0.0,onbasics
onbasics,0.0,0.0,0.350061,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.583273,0.0,0.0,onbasics
onbasics,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.937778,0.0,0.0,onbasics
onbasics,0.0,0.0,0.000000,0.328652,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.592560,0.0,0.0,onbasics


In [119]:
df.groupby('brand').mean()

,the it a to i is this and as you,tablet so for good far this no a kids with,i it was a my and the for this with,the is buy best quality good and price great battery,i not to a tablet it what do but does,you to the have a it be if will that,i kindle my it and one the to love had,the is and a fire screen to kindle this than,i the a of in and to batteries these have,to and the tablet easy use apps for is of,it for my and she loves this to old her,they and them work are for these great love kids,batteries as long last these the they brand just name,the it for is and a kids i to that,great price good for a product works and value recommend
brand,,,,,,,,,,,,,,,
on,0.058061,0.040428,0.106815,0.032203,0.052268,0.044165,0.072711,0.079582,0.023987,0.114957,0.160623,0.024013,0.007183,0.095410,0.047279
onBasics,0.137390,0.002222,0.068753,0.026464,0.084788,0.035255,0.085208,0.179564,0.244723,0.012615,0.002222,0.005485,0.023123,0.032449,0.037238
onbasics,0.026026,0.032617,0.027097,0.072135,0.037492,0.042716,0.020693,0.012597,0.176041,0.013705,0.011794,0.073187,0.241993,0.013009,0.163811


In [120]:
from gensim.models.coherencemodel import CoherenceModel

def compute_coherence_values(dictionary, corpus, df, limit, start=2, step=3, passes=5):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    path : path to input texts
    limit : Max num of topics
    passes: the number of times the entire lda model & coherence values are calculated

    Returns:
    -------
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    
    coherence_values = []
    
    tokens = [tokenize(i) for i in df['reviews.text']]
    
    for iter_ in range(passes):
        for num_topics in range(start, limit, step):
            stream = df['tokens']
            model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=2)
            coherencemodel = CoherenceModel(model=model,dictionary=dictionary,corpus=corpus, coherence='u_mass')
            coherence_values.append({'pass': iter_, 
                                     'num_topics': num_topics, 
                                     'coherence_score': coherencemodel.get_coherence()
                                    })

    return coherence_values

In [ ]:
# Can take a long time to run.
coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus, 
                                                        df=amazon, 
                                                        start=2, 
                                                        limit=40, 
                                                        step=6,
                                                        passes=40)

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling